# 목표

- 데이터를 보고 HS부호를 뽑는 프로세스를 이해하기
- 추천 시스템과 RAG 중 어떤 방법을 선택할지 정하기
- 추가할 데이터는 없는지 고려해보기

# Library

In [ ]:
import re
import numpy as np
import pandas as pd
from tqdm import tqdm

# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'.venv/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as msno

# Data Load

In [ ]:
text_data = pd.read_csv('./data/비식별된 해외기업별 영문 텍스트데이터.csv', dtype=str)
statistics_data = pd.read_csv('./data/통계청 국제표준산업분류 HSCODE 6단위 매핑.csv', dtype=str)
customs_data = pd.read_csv('./data/관세청_HS부호_240101.csv', dtype=str)

In [ ]:
text_data_copy = text_data.copy()
statistics_data_copy = statistics_data.copy()
customs_data_copy = customs_data.copy()

In [ ]:
display(text_data_copy.info())
display(statistics_data_copy.info())
display(customs_data_copy.info())

# 중복값 확인

## 텍스트

In [ ]:
text_data_copy[text_data_copy.duplicated(keep=False)]

- 결과값 컬럼 개수 때문에 중복값 제거는 안 함

## 통계청

In [ ]:
statistics_data_copy[statistics_data_copy.duplicated(keep=False)]

In [ ]:
print(statistics_data_copy.shape)
print(statistics_data_copy[statistics_data_copy.duplicated()].shape)
print(statistics_data_copy.drop_duplicates().shape)

In [ ]:
statistics_data_copy = statistics_data_copy.drop_duplicates()
statistics_data_copy = statistics_data_copy.reset_index(drop=True)

## 관세청

In [ ]:
customs_data_copy[customs_data_copy.duplicated()]

# 결측치 채우기 (보류)

- 윗줄꺼로 보간은 아닌 것 같음. 결측치를 채우기 위한 교차 확인이 되지 않음. 일단.. 결측치 있는채로 진행

In [ ]:
# 일단 공백으로 채우기
text_data_copy = text_data_copy.fillna('')
statistics_data_copy = statistics_data_copy.fillna('')
customs_data_copy = customs_data_copy.fillna('')

# 텍스트 -> 통계청, 관세청

In [ ]:
# 영문 텍스트 확인
text_data_copy.iloc[0]

In [ ]:
# 영문 텍스트 확인
text_data_copy.iloc[0, 2]

자동차 수리점, 기타 특별한 자동차 수리, 연료 서비스, 카뷰레터 수리, 브레이크 라이닝 교체, 

전면 및 휠 정렬, 라디에이터 수리 등. 자동차 수리 및 유지보수, 자동차 브레이크 라이닝 설치, 

기타 자동차 기계 및 전기 수리 및 유지보수, 자동차의 유지보수 및 수리, 자동차의 유지보수 및 수리, 기타 자동차 수리 및 유지보수.

In [ ]:
# 텍스트 데이터의 CODE를 통계청에서 확인
statistics_data_copy[statistics_data_copy['ISIC4_국제표준산업분류'] == '4520']

- 영문 텍스트를 해석하면 7630과 7631행과 관련이 있는 것으로 보임
- 관세청에서 자동차 수리 관련 행을 찾아야 할 듯

In [ ]:
customs_data_copy.head()

In [ ]:
idx_list_0 = []
idx_list_1 = []
for idx in range(0, customs_data_copy.shape[0]):
    # 먼저 한글품목명 기준으로 키워드를 찾는다.
    cond = customs_data_copy.iloc[idx, 1]
    if '자동차' in cond: # 수리, 자동차, 부품
        # print(idx, end=' ')
        idx_list_0.append(idx)
        # 한글품목명으로 걸러진 데이터에서 성질통합분류코드명 기준으로 키워드를 찾는다.
        cond2 = customs_data_copy.iloc[idx, 4]
        if '부품' in cond2:
            # print(idx, end=' ')
            idx_list_1.append(idx)

In [ ]:
customs_data_copy.iloc[idx_list_1] # idx_list_0

In [ ]:
idx_list_2 = []
for idx in range(0, customs_data_copy.shape[0]):
    # 영문품목명 기준으로 키워드를 찾는다.
    cond = customs_data_copy.iloc[idx, 2]
    if 'auto' in cond: # auto
        # print(idx, end=' ')
        idx_list_2.append(idx)

In [ ]:
customs_data_copy.iloc[idx_list_2]

In [ ]:
text_data_copy.to_csv('./data/영문_텍스트.csv', index=False, encoding='utf-8-sig')
statistics_data_copy.to_csv('./data/통계청.csv', index=False, encoding='utf-8-sig')
customs_data_copy.to_csv('./data/관세청.csv', index=False, encoding='utf-8-sig')

- 데이터를 보고 HS부호를 뽑는 프로세스를 이해하기 
    - 완료
    - 근데 관세청에서 한글과 영어 중 어떤 것이 더 좋은지 모르겠음.. 한글과 영어를 혼합해서 찾아야 할 것 같음
- 추천 시스템과 RAG 중 어떤 방법을 선택할지 정하기
    - RAG가 더 좋을 것 같음..!
- 추가할 데이터는 없는지 고려해보기
    - 포털에서 통계청, 관세청의 HS부호로 영문으로 되어있는 문장을 추가로 넣어줄 수 있을 것 같음
    - 영문 텍스트 데이터와 연관성이 더 높아질 것 같음
    - 추가 데이터로 관세청에서 HS부호를 더 적절히 추출할 수 있을 것 같음